In [1]:
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
pd.options.display.max_columns = None
from pybaseball import batting_stats, batting_stats_range, pitching_stats_range, cache, playerid_lookup, statcast_batter, statcast_pitcher, statcast
import math
import sqlite3
#from fantasy_utils import Fantasy_Projections
import fantasy_utils as fu
import time
cache.enable()
cache.config.cache_type='csv'
cache.config.save()

In [2]:
fp = fu.Fantasy_Projections()
#df = fp.make_projections(fp.yr)

In [3]:
#keepers Google doc
keepers_url = 'https://docs.google.com/spreadsheets/d/1dwDC2uMsfVRYeDECKLI0Mm_QonxkZvTkZTfBgnZo7-Q/edit#gid=1723951361'

In [3]:
from sqlalchemy import MetaData, text, Column, Integer, String, ForeignKey, Table, create_engine, Float, Boolean, DateTime
from sqlalchemy.orm import relationship, backref, sessionmaker
from sqlalchemy.ext.declarative import declarative_base

meta = MetaData()
engine = create_engine('sqlite:///fantasy_data.db', echo=False)
Session = sessionmaker(bind=engine)
session = Session()
Base = declarative_base()

In [ ]:
ids = fp.load_id_map()

## Create Tables 

### owners
Create owners table with id and name fields

In [ ]:
owner_dict = {"owner_id":[30, 3, 36, 41, 4, 27, 38, 44, 1, 29, 42, 45],
              "owner":["9 Grand Kids", 'Brewbirds', 'Charmer', 'Dirty Birds', "Harveys Wallbangers", 'Lil Trump & the Ivanabees', 'Lima Time!', 'Roid Ragers', 'Trouble with the Curve', 'Ugly Spuds', 'Wiscompton Wu-Tang', 'Young Guns'],
             }

In [ ]:
# Uncomment below line to overwrite owners table
#pd.DataFrame(owner_dict).to_sql("owners", con=engine, if_exists='append', index=False)

### roster

In [ ]:
ros = pd.read_excel('data//2023-owner-weekly-sit-start.xlsx')
owners = pd.read_sql('owners', engine)
ros = ros.merge(owners, left_on='Owner', right_on='owner', how='outer', indicator=True)
ros.loc[ros['Decision']=='Sit', 'Pos'] = 'B'
ros.loc[ros['Decision']=='Start', 'Decision'] = 1
ros.loc[ros['Decision']=='Sit', 'Decision'] = 0

In [ ]:
ros.info()

In [ ]:
ros.value_counts('_merge')

In [ ]:
pd.pivot_table(ros,index='owner', columns='Week', aggfunc='count', values='CBSID')

In [ ]:
ros[ros['Decision']==1].Pos.value_counts()

In [ ]:
ros.rename(columns={'CBSID':'cbsid', 'Year':'year', 'Week':'week', 'Pos':'pos'},inplace=True)

In [ ]:
# Uncomment below to append to roster table
ros[['cbsid', 'owner_id', 'year', 'week', 'pos']].to_sql('roster', index=False, if_exists='replace', con=engine)

In [ ]:
pd.read_sql('roster', engine)

### drafted

In [ ]:
pd.read_sql("SELECT name FROM sqlite_master", engine)

In [ ]:
#cursor.execute("CREATE TABLE players2023 AS SELECT * FROM players;").commit()

In [ ]:
pd.read_sql('players2023', engine)

In [ ]:
p = pd.read_sql('players',engine)
p.rename(columns={'Owner':'owner', 'Paid':'paid', 'Keeper':'keeper','Timestamp':'timestamp'},inplace=True)
p.rename(columns={'CBSID':'cbsid'},inplace=True)
p['year'] = 2023
p.info()

In [ ]:
p.owner.replace({'Harvey':'Harveys Wallbangers', 'Lima Time':'Lima Time!', 'Wu-Tang':'Wiscompton Wu-Tang', 'Midnight':'9 Grand Kids', 'Trouble':'Trouble with the Curve', 'Lil Trump':'Lil Trump & the Ivanabees'},inplace=True)

In [ ]:
p.merge(owners, on='owner', how='inner').groupby(['owner','owner_id'])['playerid'].count()

In [ ]:
p = p.merge(owners, on='owner', how='inner')

In [ ]:
p.loc[p['Name']=='Trevor Story', 'paid'] = 0

In [ ]:
p = p.merge(ids[['CBSNAME', 'CBSID', 'IDFANGRAPHS']], left_on='playerid', right_on='IDFANGRAPHS', how='left').sort_values('CBSID')

In [ ]:
p.loc[p['Name']=='Brice Turang', 'CBSID'] = 2942981
p.loc[p['Name']=='Matt Mervis', 'CBSID'] = 28944596
p.loc[p['Name']=='Anthony Volpe', 'CBSID'] = 3117930
p.loc[p['Name']=='Sal Frelick', 'CBSID'] = 26719606
p.loc[p['Name']=='Grayson Rodriguez', 'CBSID'] = 2942971
p.loc[p['Name']=='Jordan Walker', 'CBSID'] = 3166486
p.loc[p['Name']=='Oscar Colas', 'CBSID'] = 28837104
p.loc[p['Name']=='Elly De La Cruz', 'CBSID'] = 26912505

In [ ]:
p[p['cbsid'].isna()]

In [ ]:
p['CBSID'] = p['CBSID'].astype(int)
p['keeper'] = p['keeper'].astype(int)

In [ ]:
# Uncomment to append to drafted table
#p[['cbsid', 'owner_id','year','paid', 'timestamp', 'keeper']].to_sql('drafted', if_exists='append', index=False, con=engine)

In [ ]:
pd.read_sql('drafted', engine)

In [ ]:
pd.read_sql("SELECT * FROM drafted d INNER JOIN owners o On (d.owner_id=o.owner_id) WHERE owner='Charmer'", engine)

In [ ]:
#pd.read_sql("DELETE FROM drafted WHERE cbsid=2901324",engine)
#pd.read_sql("INSERT INTO drafted VALUES (2901324, 36, 2023, 27, '2023-10-08 09:12:55.598431', 0)", engine)

In [ ]:
p[p['cbsid']==2901324]

### eligibility

In [ ]:
df = pd.read_excel('data//2023-weekly-stats-h.xlsx')
df2 = pd.read_excel('data//2023-weekly-stats-p.xlsx')

In [ ]:
df[(df['CBSID'].notna())][['CBSID', 'all_pos']].iloc[0]['all_pos']

In [ ]:
df2['all_pos'] = "['P']"
df2.loc[df2['CBSID']==2901324, 'all_pos'] = "['DH', 'P']"

In [ ]:
df2[(df2['CBSID'].notna()) & (df2['Decision']!=0)][['Player', 'CBSID', 'CBSNAME', 'Pos', 'all_pos']]

In [ ]:
df2 = df2[(df2['CBSID']==2901324) & (df2['Decision']!=0) | (df2['CBSID']!=2901324)]

In [ ]:
df3 = pd.concat([df, df2])
df3 = df3[df3['Owner']!='FA']
df3['year'] = 2023
df3.rename(columns={'CBSID':'cbsid', 'Week':'week'}, inplace=True)

In [ ]:
df3 = df3.sort_values(['cbsid','week'])

In [ ]:
df3.loc[(df3['CBSNAME']=='Will Smith') & (df3['Owner']=='Ugly Spuds'), 'all_pos'] = "['C', 'DH']"

In [ ]:
df3['cbsid'] = df3['cbsid'].astype(int)

In [ ]:
df3.loc[df3['CBSNAME']==0, 'cbsid'] = 2120231

In [ ]:
df3[df3['CBSNAME']=='Shohei Ohtani'][['cbsid', 'CBSNAME', 'year', 'week', 'all_pos']]

In [ ]:
df3.groupby('CBSNAME')['week'].count().sort_values()

In [ ]:
df3.shape[0]/12/27

In [142]:
e = pd.read_sql('select * from eligibility', engine)

#for pos in ['C', '1B', '2B', '3B', 'SS', 'MI', 'CI', 'OF', 'DH', 'SP', 'RP', 'P']:
#    e[pos] = e.all_pos.apply(lambda x: 1 if pos in x else 0)
#e.rename(columns={i:'pos'+i for i in ['C', '1B', '2B', '3B', 'SS', 'MI', 'CI', 'OF', 'DH', 'SP', 'RP', 'P']},inplace=True)

In [144]:
e.rename(columns={i:i.replace('-','') for i in ['pos-C', 'pos-1B', 'pos-2B', 'pos-3B', 'pos-SS', 'pos-MI', 'pos-CI', 'pos-OF', 'pos-DH', 'pos-SP', 'pos-RP', 'pos-P']},inplace=True)

In [146]:
# Uncomment to append to eligibility
#e.sort_values(['cbsid', 'year', 'week']).to_sql('eligibility', if_exists='replace', con=conn, index=False)

In [147]:
pd.read_sql('select * from eligibility', engine)

,cbsid,year,week,all_pos,posC,pos1B,pos2B,pos3B,posSS,posMI,posCI,posOF,posDH,posSP,posRP,posP
0,389743,2023,1,['P'],0,0,0,0,0,0,0,0,0,0,0,1
1,389743,2023,2,['P'],0,0,0,0,0,0,0,0,0,0,0,1
2,389743,2023,3,['P'],0,0,0,0,0,0,0,0,0,0,0,1
3,389743,2023,4,['P'],0,0,0,0,0,0,0,0,0,0,0,1
4,389743,2023,5,['P'],0,0,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10685,29075539,2023,23,['P'],0,0,0,0,0,0,0,0,0,0,0,1
10686,29075539,2023,24,['P'],0,0,0,0,0,0,0,0,0,0,0,1
10687,29075539,2023,25,['P'],0,0,0,0,0,0,0,0,0,0,0,1
10688,29075539,2023,26,['P'],0,0,0,0,0,0,0,0,0,0,0,1


### stats

In [ ]:
#df = pd.read_excel('data//2023-weekly-stats-h.xlsx')
#df2 = pd.read_excel('data//2023-weekly-stats-p.xlsx')
df2 = df2[(df2['CBSID']==2901324) & (df2['Decision']!=0) | (df2['CBSID']!=2901324)]

In [ ]:
df[df['Owner']!='FA'].shape, df2[df2['Owner']!='FA'].shape

In [ ]:
df.shape, df2.shape

In [ ]:
df2.sort_values('CBSID')

#### Players

In [ ]:
dfh = pd.read_excel('data//2023-weekly-stats-h.xlsx')

In [ ]:
dfp = pd.read_excel('data//2023-weekly-stats-p.xlsx')

In [ ]:
df = pd.concat([dfh.drop_duplicates('cbsid'), dfp.drop_duplicates('cbsid')]).drop_duplicates('cbsid')

In [ ]:
df.cbsid.nunique(), df.shape

In [ ]:
df = df[['cbsid', 'CBSNAME']].merge(ids, left_on=['cbsid', 'CBSNAME'], right_on=['CBSID', 'CBSNAME'], how='outer', indicator=True)
df['cbsid'].fillna(df['CBSID'],inplace=True)
df = df[~df.index.isin([3327])]

In [ ]:
df[['cbsid', 'CBSNAME', 'IDPLAYER', 'PLAYERNAME', 'BIRTHDATE', 'FIRSTNAME','LASTNAME', 'TEAM', 'LG', 'POS', 'IDFANGRAPHS', 'FANGRAPHSNAME',
    'MLBID', 'MLBNAME', 'RETROID', 'BREFID', 'NFBCID', 'NFBCNAME', 'ESPNID', 'ESPNNAME', 'BPID', 'YAHOOID', 'YAHOONAME', 'MSTRBLLNAME', 
    'BATS', 'THROWS', 'FANTPROSNAME', 'LASTCOMMAFIRST', 'ROTOWIREID', 'FANTRAXID', 'FANTRAXNAME', 'ROTOWIRENAME', 'ALLPOS', 'NFBCLASTFIRST', 
    'ACTIVE']].to_sql('players', if_exists='replace', index=False, con=conn)

In [ ]:
pd.read_sql('SELECT * FROM players WHERE IDFANGRAPHS="29911"',engine)

In [ ]:
#cursor.execute('DELETE FROM players WHERE cbsid=2836515 AND CBSNAME="Andrew Abbott"')
#cursor.execute("UPDATE players SET cbsid=2836515 WHERE IDFANGRAPHS='29911'")
#conn.commit()

In [ ]:
cursor.execute('DELETE FROM players WHERE cbsid=2901324 AND POS="P"')

### Create Views

In [12]:
def drop_view(vw):
    cursor.execute(f"DROP VIEW {vw};")
    conn.commit()
    return f"{vw} dropped successfully"

def add_view(vw):
    cursor.execute(vw)
    conn.commit()
    return f"added view"

In [103]:
conn = sqlite3.connect('fantasy_data.db')
conn.create_function('sqrt', 1, math.sqrt)
cursor = conn.cursor()

In [6]:
pd.read_sql("SELECT name FROM sqlite_master", engine)

,name
0,hitting
1,ix_hitting_index
2,hitters
3,sqlite_autoindex_hitters_1
4,players2022
5,owners
6,drafted
7,eligibility
8,roster
9,stats


In [34]:
pd.read_sql("with hitting_stats AS ( \
    SELECT year, count(distinct cbsid) hitting_records, \
        sum(AB) AB_sum, avg(AB) AB_mean, sqrt((SUM(AB*AB) - SUM(AB)*SUM(AB)/COUNT(*)) / (COUNT(*)-1)) As AB_std, \
        sum(H) H_sum, avg(H) H_mean, sqrt((SUM(H*H) - SUM(H)*SUM(H)/COUNT(*)) / (COUNT(*)-1)) As H_std, \
        sum(R) R_sum, avg(R) R_mean, sqrt((SUM(R*R) - SUM(R)*SUM(R)/COUNT(*)) / (COUNT(*)-1)) As R_std, \
        sum(RBI) RBI_sum, avg(RBI) RBI_mean, sqrt((SUM(RBI*RBI) - SUM(RBI)*SUM(RBI)/COUNT(*)) / (COUNT(*)-1)) As RBI_std, \
        sum(HR) HR_sum, avg(HR) HR_mean, sqrt((SUM(HR*HR) - SUM(HR)*SUM(HR)/COUNT(*)) / (COUNT(*)-1)) As HR_std, \
        sum(SB) SB_sum, avg(SB) SB_mean, sqrt((SUM(SB*SB) - SUM(SB)*SUM(SB)/COUNT(*)) / (COUNT(*)-1)) As SB_std, \
        sum(H)/sum(AB) lgBA \
    FROM vw_player_totals \
    WHERE AB >= 400 \
    GROUP BY year), \
    pitching_stats As ( \
    SELECT year, count(distinct cbsid) pitching_records, \
        sum(outs) outs_sum, avg(outs) outs_mean, sqrt((SUM(outs*outs) - SUM(outs)*SUM(outs)/COUNT(*)) / (COUNT(*)-1)) As outs_std, \
        sum(IP) IP_sum, avg(IP) IP_mean, sqrt((SUM(IP*IP) - SUM(IP)*SUM(IP)/COUNT(*)) / (COUNT(*)-1)) As IP_std, \
        sum(ER) ER_sum, avg(ER) ER_mean, sqrt((SUM(ER*ER) - SUM(ER)*SUM(ER)/COUNT(*)) / (COUNT(*)-1)) As ER_std, \
        sum(Ha) Ha_sum, avg(Ha) Ha_mean, sqrt((SUM(Ha*Ha) - SUM(Ha)*SUM(Ha)/COUNT(*)) / (COUNT(*)-1)) As Ha_std, \
        sum(BBa) BBa_sum, avg(BBa) BBa_mean, sqrt((SUM(BBa*BBa) - SUM(BBa)*SUM(BBa)/COUNT(*)) / (COUNT(*)-1)) As BBa_std, \
        sum(W) W_sum, avg(W) W_mean, sqrt((SUM(W*W) - SUM(W)*SUM(W)/COUNT(*)) / (COUNT(*)-1)) As W_std, \
        sum(SO) SO_sum, avg(SO) SO_mean, sqrt((SUM(SO*SO) - SUM(SO)*SUM(SO)/COUNT(*)) / (COUNT(*)-1)) As SO_std, \
        sum(SvHld) SvHld_sum, avg(SvHld) SvHld_mean, sqrt((SUM(SvHld*SvHld) - SUM(SvHld)*SUM(SvHld)/COUNT(*)) / (COUNT(*)-1)) As SvHld_std, \
        sum(ER)/(sum(outs)/3.)*9 lgERA, (sum(Ha)+sum(BBa))/(sum(outs)/3.) lgWHIP \
    FROM vw_player_totals \
    WHERE IP > 55 AND SvHld >= 7 OR IP > 165 \
    GROUP BY year) \
    SELECT * \
    FROM hitting_stats h, pitching_stats p \
    WHERE h.year=p.year", conn)

,year,hitting_records,AB_sum,AB_mean,AB_std,H_sum,H_mean,H_std,R_sum,R_mean,R_std,RBI_sum,RBI_mean,RBI_std,HR_sum,HR_mean,HR_std,SB_sum,SB_mean,SB_std,lgBA,year,pitching_records,outs_sum,outs_mean,outs_std,IP_sum,IP_mean,IP_std,ER_sum,ER_mean,ER_std,Ha_sum,Ha_mean,Ha_std,BBa_sum,BBa_mean,BBa_std,W_sum,W_mean,W_std,SO_sum,SO_mean,SO_std,SvHld_sum,SvHld_mean,SvHld_std,lgERA,lgWHIP
0,2023,168,87131.0,518.636905,69.480918,22871.0,136.136905,24.596851,12616.0,75.095238,18.917482,12255.0,72.946429,19.831705,3486.0,20.75,9.285718,1920.0,11.428571,12.662156,0.26249,2023,148,43976.0,297.135135,163.329917,14658.75,99.045608,54.443806,5976.0,40.378378,26.510787,12779.0,86.344595,52.607869,4907.0,33.155405,16.648308,947.0,6.398649,4.274175,15483.0,104.614865,55.436244,2283.0,15.425676,12.633554,3.669092,1.206522


In [35]:
vw_lg_summary = "CREATE VIEW vw_lg_summary AS \
    with hitting_stats AS ( \
    SELECT year, count(distinct cbsid) hitting_records, \
        sum(AB) AB_sum, avg(AB) AB_mean, sqrt((SUM(AB*AB) - SUM(AB)*SUM(AB)/COUNT(*)) / (COUNT(*)-1)) As AB_std, \
        sum(H) H_sum, avg(H) H_mean, sqrt((SUM(H*H) - SUM(H)*SUM(H)/COUNT(*)) / (COUNT(*)-1)) As H_std, \
        sum(BA_cnt) BA_cnt_sum, avg(BA_cnt) BA_cnt_mean, sqrt((SUM(BA_cnt*BA_cnt) - SUM(BA_cnt)*SUM(BA_cnt)/COUNT(*)) / (COUNT(*)-1)) As BA_cnt_std, \
        sum(R) R_sum, avg(R) R_mean, sqrt((SUM(R*R) - SUM(R)*SUM(R)/COUNT(*)) / (COUNT(*)-1)) As R_std, \
        sum(RBI) RBI_sum, avg(RBI) RBI_mean, sqrt((SUM(RBI*RBI) - SUM(RBI)*SUM(RBI)/COUNT(*)) / (COUNT(*)-1)) As RBI_std, \
        sum(HR) HR_sum, avg(HR) HR_mean, sqrt((SUM(HR*HR) - SUM(HR)*SUM(HR)/COUNT(*)) / (COUNT(*)-1)) As HR_std, \
        sum(SB) SB_sum, avg(SB) SB_mean, sqrt((SUM(SB*SB) - SUM(SB)*SUM(SB)/COUNT(*)) / (COUNT(*)-1)) As SB_std, \
        sum(H)/sum(AB) lgBA \
    FROM vw_player_totals \
    WHERE AB >= 400 \
    GROUP BY year), \
    pitching_stats As ( \
    SELECT year, count(distinct cbsid) pitching_records, \
        sum(outs) outs_sum, avg(outs) outs_mean, sqrt((SUM(outs*outs) - SUM(outs)*SUM(outs)/COUNT(*)) / (COUNT(*)-1)) As outs_std, \
        sum(IP) IP_sum, avg(IP) IP_mean, sqrt((SUM(IP*IP) - SUM(IP)*SUM(IP)/COUNT(*)) / (COUNT(*)-1)) As IP_std, \
        sum(ER) ER_sum, avg(ER) ER_mean, sqrt((SUM(ER*ER) - SUM(ER)*SUM(ER)/COUNT(*)) / (COUNT(*)-1)) As ER_std, \
        sum(Ha) Ha_sum, avg(Ha) Ha_mean, sqrt((SUM(Ha*Ha) - SUM(Ha)*SUM(Ha)/COUNT(*)) / (COUNT(*)-1)) As Ha_std, \
        sum(BBa) BBa_sum, avg(BBa) BBa_mean, sqrt((SUM(BBa*BBa) - SUM(BBa)*SUM(BBa)/COUNT(*)) / (COUNT(*)-1)) As BBa_std, \
        sum(ERA_cnt) ERA_cnt_sum, avg(ERA_cnt) ERA_cnt_mean, sqrt((SUM(ERA_cnt*ERA_cnt) - SUM(ERA_cnt)*SUM(ERA_cnt)/COUNT(*)) / (COUNT(*)-1)) As ERA_cnt_std, \
        sum(WHIP_cnt) WHIP_cnt_sum, avg(WHIP_cnt) WHIP_cnt_mean, sqrt((SUM(WHIP_cnt*WHIP_cnt) - SUM(WHIP_cnt)*SUM(WHIP_cnt)/COUNT(*)) / (COUNT(*)-1)) As WHIP_cnt_std, \
        sum(W) W_sum, avg(W) W_mean, sqrt((SUM(W*W) - SUM(W)*SUM(W)/COUNT(*)) / (COUNT(*)-1)) As W_std, \
        sum(SO) SO_sum, avg(SO) SO_mean, sqrt((SUM(SO*SO) - SUM(SO)*SUM(SO)/COUNT(*)) / (COUNT(*)-1)) As SO_std, \
        sum(SvHld) SvHld_sum, avg(SvHld) SvHld_mean, sqrt((SUM(SvHld*SvHld) - SUM(SvHld)*SUM(SvHld)/COUNT(*)) / (COUNT(*)-1)) As SvHld_std, \
        sum(ER)/(sum(outs)/3.)*9 lgERA, (sum(Ha)+sum(BBa))/(sum(outs)/3.) lgWHIP \
    FROM vw_player_totals \
    WHERE IP > 55 AND SvHld >= 7 OR IP > 165 \
    GROUP BY year) \
    SELECT * \
    FROM hitting_stats h, pitching_stats p \
    WHERE h.year=p.year"

In [36]:
drop_view('vw_lg_summary')

'vw_lg_summary dropped successfully'

In [37]:
add_view(vw_lg_summary)

'added view'

In [38]:
pd.read_sql("SELECT * FROM vw_lg_summary", conn)

,year,hitting_records,AB_sum,AB_mean,AB_std,H_sum,H_mean,H_std,BA_cnt_sum,BA_cnt_mean,BA_cnt_std,R_sum,R_mean,R_std,RBI_sum,RBI_mean,RBI_std,HR_sum,HR_mean,HR_std,SB_sum,SB_mean,SB_std,lgBA,year:1,pitching_records,outs_sum,outs_mean,outs_std,IP_sum,IP_mean,IP_std,ER_sum,ER_mean,ER_std,Ha_sum,Ha_mean,Ha_std,BBa_sum,BBa_mean,BBa_std,ERA_cnt_sum,ERA_cnt_mean,ERA_cnt_std,WHIP_cnt_sum,WHIP_cnt_mean,WHIP_cnt_std,W_sum,W_mean,W_std,SO_sum,SO_mean,SO_std,SvHld_sum,SvHld_mean,SvHld_std,lgERA,lgWHIP
0,2023,168,87131.0,518.636905,69.480918,22871.0,136.136905,24.596851,1227.098271,7.304156,13.414825,12616.0,75.095238,18.917482,12255.0,72.946429,19.831705,3486.0,20.75,9.285718,1920.0,11.428571,12.662156,0.26249,2023,148,43976.0,297.135135,163.329917,14658.75,99.045608,54.443806,5976.0,40.378378,26.510787,12779.0,86.344595,52.607869,4907.0,33.155405,16.648308,9532.229652,64.406957,101.079324,1552.028903,10.486682,16.620995,947.0,6.398649,4.274175,15483.0,104.614865,55.436244,2283.0,15.425676,12.633554,3.669092,1.206522


#### Season totals by player by year

In [ ]:
vw_player_totals = "CREATE VIEW vw_player_totals As \
    with hitting_stats AS ( \
    SELECT year, cbsid, \
        SUM(AB) AB, SUM(H) H, SUM(R) R, SUM(RBI) RBI, SUM(HR) HR, SUM(SB) SB, SUM(H)/SUM(AB) BA \
    FROM stats \
    GROUP BY year, cbsid \
    ), lgBA As ( \
    SELECT year, SUM(H)/SUM(AB) lgBA FROM stats WHERE AB IS NOT NULL \
    GROUP BY year \
    ), lgPitching As ( \
    SELECT \
        year, \
        SUM(ER)/(SUM(outs)/3.)*9 lgERA, \
        (SUM(Ha)+SUM(BBa))/(SUM(outs)/3.) lgWHIP \
    FROM stats \
    WHERE IP IS NOT NULL \
    GROUP BY year \
    ), pitching_stats AS ( \
    SELECT \
        year, cbsid, SUM(outs) outs, SUM(IP) IP, SUM(W) W, SUM(SO) SO, SUM(SvHld) SvHld, SUM(ER)/(SUM(outs)/3.)*9 ERA, (SUM(Ha)+SUM(BBa))/(SUM(outs)/3.) WHIP, \
        SUM(Ha) Ha, SUM(BBa) BBa, SUM(ER) ER \
    FROM stats \
    GROUP BY year, cbsid \
    ) \
    SELECT h.*, \
            (h.H - (h.AB*lgBA)) BA_cnt, \
            p.outs, p.IP, p.W, p.SO, p.SvHld, p.ERA, p.WHIP, p.Ha, p.BBa, p.ER, \
            ((p.ER*9) - ((p.outs/3.)*lgERA))*-1 ERA_cnt, \
            ((p.Ha+p.BBa) - ((p.outs/3.)*lgWHIP))*-1 WHIP_cnt \
    FROM hitting_stats h, pitching_stats p, lgBA, lgPitching \
    WHERE h.year=p.year AND h.cbsid=p.cbsid AND lgBA.year=h.year AND lgPitching.year=h.year"

In [ ]:
pd.read_sql("with hitting_stats AS ( \
    SELECT year, cbsid, \
        SUM(AB) AB, SUM(H) H, SUM(R) R, SUM(RBI) RBI, SUM(HR) HR, SUM(SB) SB, SUM(H)/SUM(AB) BA \
    FROM stats \
    GROUP BY year, cbsid \
    ), lgBA As ( \
    SELECT year, SUM(H)/SUM(AB) lgBA FROM stats WHERE AB IS NOT NULL \
    GROUP BY year \
    ), lgPitching As ( \
    SELECT \
        year, \
        SUM(ER)/(SUM(outs)/3.)*9 lgERA, \
        (SUM(Ha)+SUM(BBa))/(SUM(outs)/3.) lgWHIP \
    FROM stats \
    WHERE IP IS NOT NULL \
    GROUP BY year \
    ), pitching_stats AS ( \
    SELECT \
        year, cbsid, SUM(outs) outs, SUM(IP) IP, SUM(W) W, SUM(SO) SO, SUM(SvHld) SvHld, SUM(ER)/(SUM(outs)/3.)*9 ERA, (SUM(Ha)+SUM(BBa))/(SUM(outs)/3.) WHIP, \
        SUM(Ha) Ha, SUM(BBa) BBa, SUM(ER) ER \
    FROM stats \
    GROUP BY year, cbsid \
    ) \
    SELECT h.*, \
            (h.H - (h.AB*lgBA)) BA_cnt, \
            p.outs, p.IP, p.W, p.SO, p.SvHld, p.ERA, p.WHIP, p.Ha, p.BBa, p.ER, \
            ((p.ER*9) - ((p.outs/3.)*lgERA))*-1 ERA_cnt, \
            ((p.Ha+p.BBa) - ((p.outs/3.)*lgWHIP))*-1 WHIP_cnt \
    FROM hitting_stats h, pitching_stats p, lgBA, lgPitching \
    WHERE h.year=p.year AND h.cbsid=p.cbsid AND lgBA.year=h.year AND lgPitching.year=h.year", conn)#.sort_values('BA_cnt', ascending=False)

# Alternate formula from PitcherList
#    Hits Above Average: (hits) – (at-bats) x (league batting average)
#    Walks plus Hits Below Average: (league WHIP) x (innings pitched) – (walks + hits)
#    Earned Runs Below Average: (league ERA/9) x (innings pitched) – (earned runs)

In [ ]:
drop_view('vw_player_totals')

In [ ]:
add_view(vw_player_totals)

In [23]:
pd.read_sql("SELECT * FROM vw_player_totals ORDER BY SB desc", conn)

,year,cbsid,AB,H,R,RBI,HR,SB,BA,BA_cnt,outs,IP,W,SO,SvHld,ERA,WHIP,Ha,BBa,ER,ERA_cnt,WHIP_cnt
0,2023,2211777,643.0,217.0,149.0,106.0,41.0,73.0,0.337481,57.274658,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023,2825651,449.0,114.0,47.0,47.0,5.0,67.0,0.253898,2.465507,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2023,3117913,565.0,161.0,116.0,76.0,25.0,54.0,0.284956,20.650360,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2023,3117472,641.0,177.0,97.0,96.0,30.0,49.0,0.276131,17.771470,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2023,3117475,563.0,138.0,83.0,64.0,18.0,47.0,0.245115,-1.852827,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,2023,29092535,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,184.0,61.33,5.0,78.0,4.0,3.228261,1.385870,61.0,24.0,22.0,66.921463,-4.506155
1456,2023,29179447,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.0,4.00,0.0,8.0,0.0,2.250000,0.250000,1.0,0.0,1.0,8.277487,4.249599
1457,2023,29188043,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.0,7.33,0.0,7.0,0.0,8.590909,3.000000,14.0,8.0,7.0,-31.324608,-12.375736
1458,2023,29190767,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39.0,13.00,0.0,12.0,1.0,6.923077,1.923077,18.0,7.0,10.0,-33.848168,-7.938805


#### League stats averages by year

In [ ]:
vw_lg_averages = "CREATE VIEW vw_lg_averages As \
    with hitting_stats AS ( \
    SELECT year, cbsid, \
        SUM(AB) AB, SUM(H) H, SUM(R) R, SUM(RBI) RBI, SUM(HR) HR, SUM(SB) SB \
    FROM stats \
    WHERE AB > 140 \
    GROUP BY year, cbsid \
    ), pitching_stats AS ( \
    SELECT \
        year, cbsid, SUM(outs) outs, SUM(IP) IP, SUM(W) W, SUM(SO) SO, SUM(SvHld) SvHld, SUM(Ha) Ha, SUM(BBa) BBa, SUM(ER) ER \
    FROM stats \
    WHERE IP > 140 OR IP BETWEEN 48 AND 90 AND SvHLD > 5 \
    GROUP BY year, cbsid \
    ) \
    SELECT \
        h.year, count(distinct h.cbsid) hitting_records, AVG(h.AB) AB, AVG(h.H) H, AVG(h.R) R, AVG(h.RBI) RBI, AVG(h.HR) HR, AVG(h.SB) SB, AVG(h.H)/AVG(h.AB) BA, \
        count(distinct p.cbsid) pitching_records, AVG(p.W) W, AVG(p.SO) SO, AVG(p.SvHld) SvHld, AVG(p.ER)/(AVG(p.outs)/3.)*9 ERA, (AVG(Ha)+AVG(BBa))/(AVG(outs)/3.) WHIP, \
        AVG(p.outs) outs, AVG(p.IP) IP, AVG(Ha) Ha, AVG(BBa) BBa, AVG(ER) ER \
    FROM hitting_stats h, pitching_stats p \
    WHERE h.year=p.year \
    GROUP BY h.year"

In [ ]:
drop_view('vw_lg_averages')

In [ ]:
add_view(vw_lg_averages)

In [87]:
pd.read_sql("SELECT * FROM vw_lg_averages", conn)

,year,hitting_records,AB,H,R,RBI,HR,SB,BA,pitching_records,W,SO,SvHld,ERA,WHIP,outs,IP,Ha,BBa,ER


In [88]:
pd.read_sql("SELECT * FROM vw_lg_summary", conn)

,year,hitting_records,AB_sum,AB_mean,AB_std,H_sum,H_mean,H_std,BA_cnt_sum,BA_cnt_mean,BA_cnt_std,R_sum,R_mean,R_std,RBI_sum,RBI_mean,RBI_std,HR_sum,HR_mean,HR_std,SB_sum,SB_mean,SB_std,lgBA,year:1,pitching_records,outs_sum,outs_mean,outs_std,IP_sum,IP_mean,IP_std,ER_sum,ER_mean,ER_std,Ha_sum,Ha_mean,Ha_std,BBa_sum,BBa_mean,BBa_std,ERA_cnt_sum,ERA_cnt_mean,ERA_cnt_std,WHIP_cnt_sum,WHIP_cnt_mean,WHIP_cnt_std,W_sum,W_mean,W_std,SO_sum,SO_mean,SO_std,SvHld_sum,SvHld_mean,SvHld_std,lgERA,lgWHIP
0,2023,134,72792.0,543.223881,54.718781,19177.0,143.11194,22.232422,1094.998248,8.171629,14.145507,10640.0,79.402985,18.361499,10210.0,76.19403,19.692553,2918.0,21.776119,9.539504,1576.0,11.761194,12.689572,0.263449,2023,185,51352.0,277.578378,157.901784,17117.32,92.526054,52.634858,6977.0,37.713514,25.513191,14878.0,80.421622,50.699515,5854.0,31.643243,16.432744,11143.12482,60.233107,95.07616,1732.782159,9.36639,15.854499,1117.0,6.037838,4.139376,18119.0,97.940541,53.853078,2763.0,14.935135,11.944896,3.668387,1.21117


#### Stats mean and StDev by year and week

In [ ]:
drop_view('vw_stats_summary')

In [ ]:
vw_stats_week_summary = "CREATE VIEW vw_stats_week_summary AS \
    with pitching_stats As (\
    SELECT year, week, count(*) pitching_rows, \
        AVG(outs) As outs_mean, \
        sqrt((SUM(outs*outs) - SUM(outs)*SUM(outs)/COUNT(*)) / (COUNT(*)-1)) As outs_std, \
        AVG(IP) As IP_mean, \
        sqrt((SUM(IP*IP) - SUM(IP)*SUM(IP)/COUNT(*)) / (COUNT(*)-1)) As IP_std, \
        AVG(W) AS W_mean, \
        sqrt((SUM(W*W) - SUM(W)*SUM(W)/COUNT(*)) / (COUNT(*)-1)) As W_std, \
        AVG(SO) AS SO_mean, \
        sqrt((SUM(SO*SO) - SUM(SO)*SUM(SO)/COUNT(*)) / (COUNT(*)-1)) As SO_std, \
        AVG(SvHld) As SvHld_mean, \
        sqrt((SUM(SvHld*SvHld) - SUM(SvHld)*SUM(SvHld)/COUNT(*)) / (COUNT(*)-1)) As SvHld_std, \
        AVG(Ha) As Ha_mean, \
        sqrt((SUM(Ha*Ha) - SUM(Ha)*SUM(Ha)/COUNT(*)) / (COUNT(*)-1)) As Ha_std, \
        AVG(BBa) As BBa_mean, \
        sqrt((SUM(BBa*BBa) - SUM(BBa)*SUM(BBa)/COUNT(*)) / (COUNT(*)-1)) As BBa_std, \
        AVG(ER) As ER_mean, \
        sqrt((SUM(ER*ER) - SUM(ER)*SUM(ER)/COUNT(*)) / (COUNT(*)-1)) As ER_std \
    FROM stats s1 \
    WHERE IP IS NOT NULL \
    GROUP BY year, week), \
    hitting_stats As ( \
    SELECT \
        year, week, count(*) hitting_rows, \
        AVG(AB) As AB_mean, \
        sqrt((SUM(AB*AB) - SUM(AB)*SUM(AB)/COUNT(*)) / (COUNT(*)-1)) As AB_std, \
        AVG(H) As H_mean, \
        sqrt((SUM(H*H) - SUM(H)*SUM(H)/COUNT(*)) / (COUNT(*)-1)) As H_std, \
        AVG(R) AS R_mean, \
        sqrt((SUM(R*R) - SUM(R)*SUM(R)/COUNT(*)) / (COUNT(*)-1)) As R_std, \
        AVG(RBI) RBI_mean, \
        sqrt((SUM(RBI*RBI) - SUM(RBI)*SUM(RBI)/COUNT(*)) / (COUNT(*)-1)) As RBI_std, \
        AVG(HR) AS HR_mean, \
        sqrt((SUM(HR*HR) - SUM(HR)*SUM(HR)/COUNT(*)) / (COUNT(*)-1)) As HR_std, \
        AVG(SB) AS SB_mean, \
        sqrt((SUM(SB*SB) - SUM(SB)*SUM(SB)/COUNT(*)) / (COUNT(*)-1)) As SB_std \
    FROM stats s1 \
    WHERE AB IS NOT NULL \
    GROUP BY year, week) \
    SELECT \
        p.*, \
        h.hitting_rows, h.AB_mean, h.AB_std, h.H_mean, h.H_std, h.R_mean, h.R_std, h.RBI_mean, h.RBI_std, h.HR_mean, h.HR_std, h.SB_mean, h.SB_std \
    FROM hitting_stats h, pitching_stats p WHERE h.year = p.year AND h.week = p.week"

In [ ]:
add_view(vw_stats_week_summary)

#### Z Table Players Season Total by Year

In [97]:
pd.read_sql("\
    SELECT \
        tot.year, \
        cbsid, \
        (R-R_mean)/R_std zR, \
        (RBI-RBI_mean)/RBI_std zRBI, \
        (HR-HR_mean)/HR_std zHR, \
        (SB-SB_mean)/SB_std zSB, \
        BA_cnt/BA_cnt_std zBA, \
        ((R-R_mean)/R_std) + ((RBI-RBI_mean)/RBI_std) + ((HR-HR_mean)/HR_std) + ((SB-SB_mean)/SB_std) + (BA_cnt/BA_cnt_std) zHit, \
        (W-W_mean)/W_std zW, \
        (SO-SO_mean)/SO_std zSO, \
        (SvHld-SvHld_mean)/SvHld_std zSvHld, \
        ERA_cnt/ERA_cnt_std zERA, \
        WHIP_cnt/WHIP_cnt_std zWHIP, \
        ((W-W_mean)/W_std) + ((SO-SO_mean)/SO_std) + ((SvHld-SvHld_mean)/SvHld_std) + (ERA_cnt/ERA_cnt_std) + (WHIP_cnt/WHIP_cnt_std) zPitch, \
        COALESCE(((R-R_mean)/R_std) + ((RBI-RBI_mean)/RBI_std) + ((HR-HR_mean)/HR_std) + ((SB-SB_mean)/SB_std) + (BA_cnt/BA_cnt_std),0) + COALESCE(((W-W_mean)/W_std) + ((SO-SO_mean)/SO_std) + ((SvHld-SvHld_mean)/SvHld_std) + (ERA_cnt/ERA_cnt_std) + (WHIP_cnt/WHIP_cnt_std),0) z, \
        SvHld \
    FROM vw_player_totals tot \
    INNER JOIN vw_lg_summary lg On tot.year=lg.year \
    ORDER BY z desc", conn)

,year,cbsid,zR,zRBI,zHR,zSB,zBA,zHit,zW,zSO,zSvHld,zERA,zWHIP,zPitch,z,SvHld
0,2023,2211777,3.790378,1.513566,2.015187,4.825916,4.048965,16.194010,NaN,NaN,NaN,NaN,NaN,NaN,16.194010,NaN
1,2023,2901324,1.230674,0.954979,2.329668,0.649258,1.947049,7.111628,0.957188,1.282368,-1.250336,1.547781,2.033287,4.570288,11.681917,0.0
2,2023,1893753,NaN,NaN,NaN,NaN,NaN,NaN,2.165100,2.303665,-1.250336,3.720688,4.370465,11.309581,11.309581,0.0
3,2023,2044509,2.592218,3.189326,3.377941,-0.848034,1.482371,9.793821,NaN,NaN,NaN,NaN,NaN,NaN,9.793821,NaN
4,2023,1630079,2.810065,1.310443,0.757260,0.885673,3.730166,9.493607,NaN,NaN,NaN,NaN,NaN,NaN,9.493607,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,2023,3117932,-4.106581,-3.716838,-2.177904,-0.926839,-0.717289,-11.645451,NaN,NaN,NaN,NaN,NaN,NaN,-11.645451,NaN
1456,2023,3158300,-4.215505,-3.818399,-2.282731,-0.690425,-0.647947,-11.655007,NaN,NaN,NaN,NaN,NaN,NaN,-11.655007,NaN
1457,2023,2211193,-4.324428,-3.818399,-2.282731,-0.926839,-0.350765,-11.703163,NaN,NaN,NaN,NaN,NaN,NaN,-11.703163,NaN
1458,2023,3152105,-4.269966,-3.869180,-2.282731,-0.926839,-0.385436,-11.734153,NaN,NaN,NaN,NaN,NaN,NaN,-11.734153,NaN


In [98]:
vw_players_season_z = "CREATE VIEW vw_players_season_z As \
    SELECT \
        tot.year, \
        cbsid, \
        (R-R_mean)/R_std zR, \
        (RBI-RBI_mean)/RBI_std zRBI, \
        (HR-HR_mean)/HR_std zHR, \
        (SB-SB_mean)/SB_std zSB, \
        BA_cnt/BA_cnt_std zBA, \
        ((R-R_mean)/R_std) + ((RBI-RBI_mean)/RBI_std) + ((HR-HR_mean)/HR_std) + ((SB-SB_mean)/SB_std) + (BA_cnt/BA_cnt_std) zHit, \
        (W-W_mean)/W_std zW, \
        (SO-SO_mean)/SO_std zSO, \
        (SvHld-SvHld_mean)/SvHld_std zSvHld, \
        ERA_cnt/ERA_cnt_std zERA, \
        WHIP_cnt/WHIP_cnt_std zWHIP, \
        ((W-W_mean)/W_std) + ((SO-SO_mean)/SO_std) + ((SvHld-SvHld_mean)/SvHld_std) + (ERA_cnt/ERA_cnt_std) + (WHIP_cnt/WHIP_cnt_std) zPitch, \
        COALESCE(((R-R_mean)/R_std) + ((RBI-RBI_mean)/RBI_std) + ((HR-HR_mean)/HR_std) + ((SB-SB_mean)/SB_std) + (BA_cnt/BA_cnt_std),0) + COALESCE(((W-W_mean)/W_std) + ((SO-SO_mean)/SO_std) + ((SvHld-SvHld_mean)/SvHld_std) + (ERA_cnt/ERA_cnt_std) + (WHIP_cnt/WHIP_cnt_std),0) z, \
        SvHld \
    FROM vw_player_totals tot \
    INNER JOIN vw_lg_summary lg On tot.year=lg.year \
    ORDER BY z desc"

In [99]:
drop_view('vw_players_season_z')

'vw_players_season_z dropped successfully'

In [100]:
add_view(vw_players_season_z)

'added view'

In [39]:
pd.read_sql("SELECT z.cbsid, p.CBSNAME player, o.owner, d.paid, ROUND(z*4.18,1) value, ROUND(z*4.18-d.paid,1) surplus, ROUND(z,2) z, \
            R, RBI, HR, SB, AB, BA_cnt, BA, IP, W, SO, t.SvHld, ERA, ERA_cnt, WHIP, WHIP_cnt, zR, zRBI, zHR, zSB, zBA \
        FROM vw_players_season_z z \
        LEFT JOIN players p On (z.cbsid=p.cbsid) \
        INNER JOIN vw_player_totals t On (z.cbsid=t.cbsid) \
        LEFT JOIN drafted d On (z.cbsid=d.cbsid) \
        LEFT JOIN owners o On (d.owner_id=o.owner_id) \
        WHERE z.year=2023 AND IP IS NULL \
        ORDER BY z desc", conn).head(10)#.z.sum()

,cbsid,player,owner,paid,value,surplus,z,R,RBI,HR,SB,AB,BA_cnt,BA,IP,W,SO,SvHld,ERA,ERA_cnt,WHIP,WHIP_cnt,zR,zRBI,zHR,zSB,zBA
0,2211777,Ronald Acuna,Wiscompton Wu-Tang,46.0,70.6,24.6,16.89,149.0,106.0,41.0,73.0,643.0,57.274658,0.337481,None,None,None,None,None,None,None,None,3.906691,1.666703,2.180768,4.862634,4.269505
1,2044509,Matt Olson,Lil Trump & the Ivanabees,26.0,43.4,17.4,10.39,127.0,139.0,54.0,1.0,608.0,20.968883,0.282895,None,None,None,None,None,None,None,None,2.743746,3.330706,3.580768,-0.823602,1.563113
2,1630079,Freddie Freeman,Brewbirds,34.0,42.5,8.5,10.16,131.0,102.0,29.0,23.0,637.0,52.765096,0.331240,None,None,None,None,None,None,None,None,2.955190,1.465006,0.888461,0.913859,3.933342
3,2106654,Mookie Betts,Charmer,36.0,38.1,2.1,9.11,126.0,107.0,39.0,14.0,584.0,33.930638,0.306507,None,None,None,None,None,None,None,None,2.690885,1.717128,1.965384,0.203080,2.529339
4,3117913,Corbin Carroll,Harveys Wallbangers,25.0,32.1,7.1,7.68,116.0,76.0,25.0,54.0,565.0,20.650360,0.284956,None,None,None,None,None,None,None,None,2.162273,0.153974,0.457692,3.362099,1.539369
5,3117472,Bobby Witt,Trouble with the Curve,30.0,31.8,1.8,7.61,97.0,96.0,30.0,49.0,641.0,17.771470,0.276131,None,None,None,None,None,None,None,None,1.157911,1.162460,0.996153,2.967222,1.324763
6,2825528,Julio Rodriguez,Harveys Wallbangers,0.0,31.3,31.3,7.48,102.0,103.0,32.0,37.0,654.0,17.542187,0.275229,None,None,None,None,None,None,None,None,1.422217,1.515431,1.211538,2.019516,1.307672
7,2184352,Kyle Tucker,Lima Time!,37.0,29.3,-7.7,7.00,97.0,112.0,29.0,30.0,574.0,20.414702,0.283972,None,None,None,None,None,None,None,None,1.157911,1.969249,0.888461,1.466688,1.521802
8,1947827,Marcus Semien,Trouble with the Curve,26.0,26.4,0.4,6.32,122.0,100.0,29.0,14.0,670.0,18.567684,0.276119,None,None,None,None,None,None,None,None,2.479440,1.364158,0.888461,0.203080,1.384117
9,2211186,Austin Riley,Brewbirds,35.0,25.4,-9.6,6.08,117.0,97.0,37.0,3.0,636.0,21.013503,0.281447,None,None,None,None,None,None,None,None,2.215134,1.212885,1.749999,-0.665651,1.566439


In [ ]:
#conv = (self.tm_dollars/self.tm_players)*(self.tot_players/(h[h['z']>0]['z'].sum()+p[p['z']>0]['z'].sum()))
(260/23)*((1465)/2234.64)

In [71]:
print(f"{round(260/23,2)} dollars per drafted player")
print(f"{23*12} total drafted players")
(260/23)*(276/506)

11.3 dollars per drafted player
276 total drafted players


6.16600790513834

In [ ]:
#Could try dividing top z post season by top z pre season and diving all z scores by that number (1.95)

In [107]:
pd.read_sql("SELECT name FROM sqlite_master", engine)

,name
0,hitting
1,ix_hitting_index
2,hitters
3,sqlite_autoindex_hitters_1
4,players2022
5,owners
6,drafted
7,roster
8,stats
9,vw_stats_week_summary


In [164]:
pd.read_sql('SELECT * FROM players2023', conn)#.to_clipboard()

,index,playerid,AB,G,H,HR,PA,R,RBI,SB,Name,Team,Pos,Primary_Pos,Dollars,CBS,sorter,BA,zR,zHR,zRBI,zSB,zBA,BIGAA,Pos_adj,z,Value,BB,ER,GS,HA,HLD,IP,SO,SV,Sv+Hld,W,ERA,WHIP,zSO,zW,zSv+Hld,zERA,zWHIP,Outs,K/9,Used,Value-CBS,Owner,Paid,Timestamp,Keeper
0,0,15640,551.0,150,155.0,44.0,655.0,106.0,109.0,9.0,Aaron Judge,NYY,OF/DH,OF,40.820435,41.0,423.75,0.280924,2.074951,2.674335,2.102390,0.137249,0.974740,7.963665,1.632999,9.596664,47.819764,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,6.819764,Roid Ragers,27,2023-10-13 20:02:40.109133,0.0
1,1,19611,580.0,151,168.0,36.0,656.0,96.0,105.0,5.0,Vladimir Guerrero,TOR,1B/DH,1B,38.588283,34.0,411.50,0.289281,1.515977,1.846890,1.904972,-0.352625,1.361739,6.276953,2.086855,8.363809,41.676500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,7.676500,Roid Ragers,37,2023-10-08 09:15:17.860783,0.0
2,2,18401,540.0,141,146.0,30.0,630.0,102.0,80.0,32.0,Ronald Acuna,ATL,OF/DH,OF,38.480032,39.0,391.50,0.271759,1.851361,1.226307,0.671108,2.954024,0.545850,7.248650,1.632999,8.881649,44.256876,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,5.256876,Wu-Tang,46,2023-10-08 09:12:32.298846,0.0
3,3,23697,566.0,145,155.0,29.0,629.0,92.0,83.0,24.0,Julio Rodriguez,SEA,OF,OF,35.327956,40.0,385.25,0.274735,1.292387,1.122876,0.819172,1.974276,0.708735,5.917447,1.632999,7.550445,37.623544,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,-2.376456,Harvey,0,2023-10-08 10:36:17.203947,0.0
4,4,20123,522.0,151,145.0,30.0,664.0,106.0,89.0,8.0,Juan Soto,SDP,OF,OF,34.649926,37.0,380.25,0.277512,2.074951,1.226307,1.115299,0.014781,0.795727,5.227064,1.632999,6.860063,34.183398,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,-2.816602,Brewbirds,34,2023-10-08 09:14:08.962121,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1168,1168,15038,NaN,30,NaN,NaN,NaN,NaN,NaN,NaN,German Marquez,COL,SP,SP,-20.696189,0.0,164.00,NaN,NaN,NaN,NaN,NaN,NaN,-3.921438,1.863028,-2.058410,-10.256969,60.0,95.0,30.0,189.0,0.0,179.175,154.0,0.0,0.0,9.0,4.771871,1.389703,0.974617,0.541989,-1.05048,-2.385320,-2.002244,537.525,7.735454,None,-10.256969,None,0,None,0.0
1169,1169,19268,NaN,17,NaN,NaN,NaN,NaN,NaN,NaN,Justin Dunn,CIN,SP,SP,-25.423594,0.0,70.00,NaN,NaN,NaN,NaN,NaN,NaN,-6.257778,1.863028,-4.394750,-21.898851,42.0,55.0,17.0,90.0,0.0,84.175,66.0,0.0,0.0,3.0,5.880606,1.568162,-0.684274,-0.823020,-1.05048,-2.009529,-1.690474,252.525,7.056727,None,-21.898851,None,0,None,0.0
1170,1170,15488,NaN,24,NaN,NaN,NaN,NaN,NaN,NaN,Antonio Senzatela,COL,SP,SP,-27.066684,0.0,83.75,NaN,NaN,NaN,NaN,NaN,NaN,-5.934292,1.863028,-4.071265,-20.286936,35.0,75.0,24.0,161.0,0.0,132.250,77.0,0.0,0.0,6.0,5.103970,1.482042,-0.476913,-0.140516,-1.05048,-2.178945,-2.087438,396.750,5.240076,None,-20.286936,None,0,None,0.0
1171,1171,11589,NaN,22,NaN,NaN,NaN,NaN,NaN,NaN,Jose Urena,COL,SP,SP,-27.475370,0.0,75.75,NaN,NaN,NaN,NaN,NaN,NaN,-7.088081,1.863028,-5.225053,-26.036215,46.0,70.0,21.0,132.0,0.0,111.075,71.0,0.0,0.0,4.0,5.671843,1.602521,-0.590019,-0.595519,-1.05048,-2.430856,-2.421207,333.225,5.752870,None,-26.036215,None,0,None,0.0


In [122]:
units_of_value = 506 #650
((260*12)-(23*12)) / units_of_value

5.620553359683795

In [126]:
pd.read_excel('data//2023-owner-weekly-sit-start.xlsx')

,CBSID,CBSNAME,Owner,Year,Week,Decision,Pos,Player,Team,MLBID,IDFANGRAPHS,Order,Auction
0,3117469,Adley Rutschman,Lima Time!,2023,1,Start,C,Adley Rutschman,BAL,668939,26288,1.0,2.0
1,26747031,Vinnie Pasquantino,Lima Time!,2023,1,Start,1B,Vinnie Pasquantino,KC,686469,27676,2.0,19.0
2,2106304,Max Muncy,Lima Time!,2023,1,Start,2B,Max Muncy,LAD,571970,13301,3.0,13.0
3,2942960,Alec Bohm,Lima Time!,2023,1,Start,3B,Alec Bohm,PHI,664761,21618,4.0,7.0
4,1945481,Xander Bogaerts,Lima Time!,2023,1,Start,SS,Xander Bogaerts,SD,593428,12161,5.0,20.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10687,1622557,David Robertson,Young Guns,2023,27,Start,RP,David Robertson,MIA,502085,8241,887.0,NaN
10688,3117538,Alek Manoah,Young Guns,2023,27,Sit,SP,Alek Manoah,TOR,666201,26410,888.0,NaN
10689,2117131,Luis Severino,Young Guns,2023,27,Sit,SP,Luis Severino,NYY,622663,15890,889.0,NaN
10690,26608291,Tyler Wells,Young Guns,2023,27,Sit,SP,Tyler Wells,BAL,669330,20000,890.0,NaN


In [160]:
e = pd.read_sql("WITH cte As (\
        SELECT e.cbsid, MAX(p.CBSNAME) name, MAX(e.week) maxWeek, max(e.pos1B) pos1B, max(pos2B) pos2B, max(pos3B) pos3B, max(posSS) posSS, max(posMI) posMI, max(posCI) posCI, \
            max(posOF) posOF, max(posDH) posDH, max(posSP) posSP, max(posRP) posRP, max(posP) posP \
        FROM eligibility e \
        INNER JOIN players p on (e.cbsid=p.cbsid) \
        GROUP BY e.cbsid) \
        SELECT cte.* FROM cte INNER JOIN eligibility e On (cte.cbsid=e.cbsid AND cte.maxWeek=e.week)", engine)

In [161]:
e

,cbsid,name,maxWeek,pos1B,pos2B,pos3B,posSS,posMI,posCI,posOF,posDH,posSP,posRP,posP
0,389743,Adam Wainwright,11,0,0,0,0,0,0,0,0,0,0,1
1,479065,Cole Hamels,20,0,0,0,0,0,0,0,0,0,0,1
2,530355,Nelson Cruz,4,0,0,0,0,0,0,0,1,0,0,0
3,530362,Justin Verlander,27,0,0,0,0,0,0,0,0,0,0,1
4,533001,Jason Hammel,1,0,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
556,28944596,Matt Mervis,10,1,0,0,0,0,1,0,1,0,0,0
557,29041593,Masataka Yoshida,27,0,0,0,0,0,0,1,1,0,0,0
558,29046609,Kodai Senga,27,0,0,0,0,0,0,0,0,0,0,1
559,29075489,Bryan Woo,27,0,0,0,0,0,0,0,0,0,0,1
